# **EDA Using SQL**

-----
### Imports

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pandasql import sqldf

------
## Read in the Data

In [62]:
reviews = pd.read_csv('../data/reviews_cleaned.csv', low_memory=False)
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3781,37776825.0,2015-07-10,36059247.0,Greg,The apartment was as advertised and Frank was ...
1,3781,41842494.0,2015-08-09,10459388.0,Tai,It was a pleasure to stay at Frank's place. Th...
2,3781,45282151.0,2015-09-01,12264652.0,Damien,The apartment description is entirely faithful...
3,3781,49022647.0,2015-09-30,41426327.0,Mike,Thoroughly enjoyed my time at Frank's home. Ha...
4,3781,52503327.0,2015-10-30,15151513.0,Ivan,Great value for the money! This location has e...


In [63]:
reviews.dtypes

listing_id        object
id               float64
date              object
reviewer_id      float64
reviewer_name     object
comments          object
dtype: object

In [64]:
# ensuring date column is correct data type (wasn't retained when csv read in)
pd.to_datetime(reviews['date'], errors = 'coerce')

0        2015-07-10
1        2015-08-09
2        2015-09-01
3        2015-09-30
4        2015-10-30
            ...    
230276   2022-05-12
230277   2022-05-22
230278   2022-05-29
230279   2022-06-08
230280   2022-06-10
Name: date, Length: 230281, dtype: datetime64[ns]

In [65]:
reviews.dtypes

listing_id        object
id               float64
date              object
reviewer_id      float64
reviewer_name     object
comments          object
dtype: object

In [66]:
listings = pd.read_csv('../data/listings_cleaned.csv')
listings.head(2)

,id,listing_url,scrape_id,last_scraped,name,description,picture_url,host_id,host_url,host_name,...,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,bathrooms,bathrooms_shared,amenities_count,host_verifications_count
0,3781,https://www.airbnb.com/rooms/3781,20210919145601,2021-09-19,HARBORSIDE-Walk to subway,Fully separate apartment in a two apartment bu...,https://a0.muscache.com/pictures/24670/b2de044...,4804,https://www.airbnb.com/users/show/4804,Frank,...,0,1,1,0,0,0.29,1.0,0,31,3
1,5506,https://www.airbnb.com/rooms/5506,20210919145601,2021-09-19,** Private! Minutes to center!**,"Private guest room with private bath, You do n...",https://a0.muscache.com/pictures/miso/Hosting-...,8229,https://www.airbnb.com/users/show/8229,Terry,...,0,10,10,0,0,0.90,1.0,0,30,4


In [67]:
listings.dtypes

id                                                int64
listing_url                                      object
scrape_id                                         int64
last_scraped                                     object
name                                             object
description                                      object
picture_url                                      object
host_id                                           int64
host_url                                         object
host_name                                        object
host_since                                       object
host_response_time                                int64
host_response_rate                              float64
host_is_superhost                                 int64
abnb_host_listings_count                        float64
host_verifications                               object
host_has_profile_pic                              int64
host_identity_verified                          

In [68]:
# converting date columns to date time
# and breaking out by year and month
# for analysis
listings['last_scraped'] = pd.to_datetime(listings['last_scraped'],
                                          format = '%Y-%m-%d',
                                          errors = 'coerce')

In [69]:
listings['last_scraped_year'] = listings['last_scraped'].dt.year 
listings['last_scraped_month'] = listings['last_scraped'].dt.month
listings[['last_scraped_year','last_scraped_month']].head(3)

,last_scraped_year,last_scraped_month
0,2021,9
1,2021,9
2,2021,9


In [70]:
listings['host_since'] = pd.to_datetime(listings['host_since'],
                                          format = '%Y-%m-%d',
                                          errors = 'coerce')
listings['host_since_year'] = listings['host_since'].dt.year 
listings['host_since_month'] = listings['host_since'].dt.month

In [71]:
listings['calendar_last_scraped'] = pd.to_datetime(listings['calendar_last_scraped'],
                                          format = '%Y-%m-%d',
                                          errors = 'coerce')
listings['calendar_last_scraped_year'] = listings['calendar_last_scraped'].dt.year 
listings['calendar_last_scraped_month'] = listings['calendar_last_scraped'].dt.month

In [72]:
listings['first_review'] = pd.to_datetime(listings['first_review'],
                                          format = '%Y-%m-%d',
                                          errors = 'coerce')
listings['first_review_year'] = listings['first_review'].dt.year 
listings['first_review_month'] = listings['first_review'].dt.month

In [73]:
listings['last_review'] = pd.to_datetime(listings['last_review'],
                                          format = '%Y-%m-%d',
                                          errors = 'coerce')
listings['last_review_year'] = listings['last_review'].dt.year 
listings['last_review_month'] = listings['last_review'].dt.month

In [85]:
listings.dtypes

id                                      int64
listing_url                            object
scrape_id                               int64
last_scraped                   datetime64[ns]
name                                   object
                                    ...      
calendar_last_scraped_month             int64
first_review_year                       int64
first_review_month                      int64
last_review_year                        int64
last_review_month                       int64
Length: 64, dtype: object

------
## Querying and Joining tables

In [88]:
sql = """
SELECT *
FROM listings
LIMIT 5
"""

df = sqldf(sql)
df.head()

,id,listing_url,scrape_id,last_scraped,name,description,picture_url,host_id,host_url,host_name,...,last_scraped_year,last_scraped_month,host_since_year,host_since_month,calendar_last_scraped_year,calendar_last_scraped_month,first_review_year,first_review_month,last_review_year,last_review_month
0,3781,https://www.airbnb.com/rooms/3781,20210919145601,2021-09-19 00:00:00.000000,HARBORSIDE-Walk to subway,Fully separate apartment in a two apartment bu...,https://a0.muscache.com/pictures/24670/b2de044...,4804,https://www.airbnb.com/users/show/4804,Frank,...,2021,9,2008,12,2021,9,2015,10,2021,7
1,5506,https://www.airbnb.com/rooms/5506,20210919145601,2021-09-19 00:00:00.000000,** Private! Minutes to center!**,"Private guest room with private bath, You do n...",https://a0.muscache.com/pictures/miso/Hosting-...,8229,https://www.airbnb.com/users/show/8229,Terry,...,2021,9,2009,2,2021,9,2011,10,2021,7
2,6695,https://www.airbnb.com/rooms/6695,20210919145601,2021-09-19 00:00:00.000000,Home Away from Home! Condo,"Comfortable, Fully Equipped private apartment...",https://a0.muscache.com/pictures/38ac4797-e7a4...,8229,https://www.airbnb.com/users/show/8229,Terry,...,2021,9,2009,2,2021,9,2014,8,2019,11
3,8789,https://www.airbnb.com/rooms/8789,20210919145601,2021-09-19 00:00:00.000000,Curved Glass Studio/1bd facing Park,This unit is for sale. There will need to be o...,https://a0.muscache.com/pictures/32210/7fdd2fd...,26988,https://www.airbnb.com/users/show/26988,Anne,...,2021,9,2009,7,2021,9,2014,12,2019,9
4,10730,https://www.airbnb.com/rooms/10730,20210919145601,2021-09-19 00:00:00.000000,Bright 1bed facing Golden Dome,"Bright, spacious unit, new galley kitchen, new...",https://a0.muscache.com/pictures/miso/Hosting-...,26988,https://www.airbnb.com/users/show/26988,Anne,...,2021,9,2009,7,2021,9,2015,3,2020,4


In [89]:
sql = """
SELECT *
FROM reviews
LIMIT 5
"""

df = sqldf(sql)
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3781,37776825.0,2015-07-10,36059247.0,Greg,The apartment was as advertised and Frank was ...
1,3781,41842494.0,2015-08-09,10459388.0,Tai,It was a pleasure to stay at Frank's place. Th...
2,3781,45282151.0,2015-09-01,12264652.0,Damien,The apartment description is entirely faithful...
3,3781,49022647.0,2015-09-30,41426327.0,Mike,Thoroughly enjoyed my time at Frank's home. Ha...
4,3781,52503327.0,2015-10-30,15151513.0,Ivan,Great value for the money! This location has e...
